In [ ]:
%%writefile seq.cpp
#include <iostream>
#include <vector>
#include <math.h>
#include <assert.h>
#include <chrono>
#include <set>

using namespace std::chrono;

// Minimal app to run a bunch of small tasks

// The following functions and variable must be defined elsewhere
extern void create_result(int work_item,std::vector<int>& result);
extern void apply_result(int work_item,const std::vector<int>& result);
extern void finish_up();
extern int work_to_be_done;

int main() {
    auto start = high_resolution_clock::now();

    // Iterate through tasks
    for(int j=0;j<work_to_be_done;j++) {
        std::vector<int> result;
        create_result(j, result);
        apply_result(j, result);
    }

    auto finish = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(finish - start);
    std::cout << "duration: " << (duration.count()*1e-6) << std::endl;

    finish_up();
}

In [ ]:
%%writefile pbm.hpp
#ifndef PBM_HPP
#define PBM_HPP

// Header for generating PBM image files
// https://en.wikipedia.org/wiki/Netpbm

#include <fstream>
#include <vector>

const int max_color = 256;

// convert an rgb tuple to an int
int make_color(int r,int g,int b) {
    assert(0 <= r && r < max_color);
    assert(0 <= g && g < max_color);
    assert(0 <= b && b < max_color);
    return (b*max_color+g)*max_color+r;
}

class PBM {
    // width and height
    int w, h;
    // a vector of vector if ints to store the pixels
    std::vector<std::vector<int>> values;
public:

    int width() { return w; }
    
    int height() { return h; }
    
    PBM(int w_, int h_) : w(w_), h(h_) {
        // initialize vector to all zeros
        for(int j=0;j<h;j++) {
            std::vector<int> row;
            for(int i=0;i<w;i++) {
                row.push_back(0);
            }
            values.push_back(std::move(row));
        }
    }
    ~PBM() { }
    
    // get or set a pixel at i, j
    int& operator()(int i,int j) {
        return values.at(j).at(i);
    }
    
    // get or set a row at j
    std::vector<int>& row(int j) {
        return values.at(j);
    }
    
    // save to a file
    void save(const char *fname) {
        std::ofstream f(fname);
        f << "P3\n";
        f << w << " " << h << "\n";
        f << "255\n";
        const int mv = 256;
        for(int j=0;j<h;j++) {
            auto& row = values.at(j);
            for(int i=0;i<w;i++) {
                int color = row.at(i);
                for(int c=0;c < 3;c++) {
                    f << (color % max_color) << " ";
                    color /= max_color;
                }
                f << "\n";
            }
        }
        f.close();
    }
};
#endif

In [ ]:
%%writefile pbm3.cpp
#include <iostream>
#include <vector>
#include <complex>
#include <math.h>
#include <assert.h>
#include <string.h>
#include "pbm.hpp"

/// For debugging, print out a vector of int's
std::ostream& operator<<(std::ostream& o,const std::vector<int>& v) {
    o << "[";
    for(int i=0;i<v.size();i++) {
        if(i > 0) o << ", ";
        o << v[i];
    }
    return o << "]";
}

const double pi = 4.0*atan(1.0);

const int max_iter = 255;

// Used to initialize Julia sets
double values[2] = {0, 0};

typedef std::complex<double> creal;

// Mandelbrot pixel creator
int pixel1(creal c) {
    creal z(0,0);
    for(int i=0;i<max_iter;i++) {
        z = z*z + c;
        if(abs(z) > 2.0) {
            return i;
        }
    }
    return max_iter;
}

// Julia pixel creator
int pixel2(creal z) {

    // If you want to play around and
    // make interesting images, change
    // this constant.
    // creal c(-0.4,.6);
    creal c(values[0], values[1]);
    
    for(int i=0;i<max_iter;i++) {
        z = z*z + c;
        if(abs(z) > 2.0) {
            return i;
        }
    }
    return max_iter;
}

const int sz = 1024*4;
int work_to_be_done = sz;
extern int work_to_be_done;

// Create a pbm image object
PBM p(sz, sz);

int (*pixel)(creal c) = pixel1;

// Create a row of the Mandelbrot/Julia set
void create_result(int row,std::vector<int>& result) {
    int h = p.height();
    int w = p.width();
    creal c = creal(0,4)*creal(row,0)/creal(h,0)-creal(0,2);
    static bool init = true;
    if(init) {
        init = false;
        const char *val1 = getenv("VAL1");
        if(val1 != nullptr) {
            values[0] = std::stod(val1);
            pixel = pixel2;
        }
        const char *val2 = getenv("VAL2");
        if(val2 != nullptr) {
            values[1] = std::stod(val2);
            pixel = pixel2;
        }
    }
    for(int i=0;i<w;i++) {
        int r=0, g=0, b=0;
        
        // Use either pixel1 or pixel2 here depending
        // on what you want.
        int iters = pixel(c + 4.0*i/w - 2.0);
        
        // Map an integer to a color
        r = (max_color-1)*sin(iters*pi/max_iter);
        g = r/2; //(max_color-1)*cos(iters*pi*.5/max_iter);
        b = (max_color-1)*cos(0.5*pi-iters*pi*.5/max_iter);
        
        result.push_back(make_color(r,g,b));
    }
}

void apply_result(int row_index,const std::vector<int>& result) {
    p.row(row_index) = result;
}

void finish_up() {
    p.save("image.pbm");
}

In [ ]:
%%writefile seq.mk
image.png : image.pbm
	convert image.pbm image.png

seq.o : seq.cpp
	g++ -std=c++14 -c seq.cpp

pbm3.o : pbm3.cpp pbm.hpp
	g++ -std=c++14 -c pbm3.cpp

seq : seq.o pbm3.o
	g++ -std=c++14 -o seq seq.o pbm3.o

image.pbm : ./seq .ALWAYS
	./seq

.ALWAYS:

In [ ]:
%%bash
make -f seq.mk

In [ ]:
%png image.png

In [ ]:
%%writefile easy.cpp
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <iostream>
#include <vector>
#include <math.h>
#include <assert.h>
#include <chrono>
#include <set>

using namespace std::chrono;

// Minimal HPX Distributed App

// The following functions and variable must be defined elsewhere
extern void create_result(int work_item,std::vector<int>& result);
extern void apply_result(int work_item,const std::vector<int>& result);
extern void finish_up();
extern int work_to_be_done;

// The basic unit of work. Does nothing but hand
// args back aa a std::vector<int>.
std::vector<int> do_work(int loc_index, int item_index) {
    std::vector<int> result;
    create_result(item_index,result);
    result.push_back(loc_index);
    result.push_back(item_index);
    return std::move(result);
}

HPX_PLAIN_ACTION(do_work, do_work_action);

int main() {
    auto locs = hpx::find_all_localities();

    // The supervisor
    auto here = hpx::find_here();

    // Assert that the supervisor is first in the list
    assert(locs[0] == here);

    // Compute the number of worker localities
    const int num_workers = locs.size() - 1;

    // Make sure we have some workers
    assert(num_workers > 0);

    // The logic below assumes more work than workers
    assert(num_workers < work_to_be_done);

    // Place to hold futures for current worker tasks
    std::vector<hpx::shared_future<std::vector<int>>> data;

    // Which items of work have been done
    std::set<int> completed;

    auto start = high_resolution_clock::now();

    // Load up workers...
    for(int j=0;j<work_to_be_done;j++) {
        int loc_index = 1 + (j % num_workers);
        auto loc = locs.at(loc_index);
        data.push_back(hpx::async<do_work_action>(loc, loc_index, j));
    }

    // Wait for remaining work to be done...
    for(int j=0;j<data.size();j++) {
        std::vector<int> result = data.at(j).get();

        // Get the item of work
        int work_item_index = result.back();
        result.pop_back();

        // Get the index of the worker that completed this task
        int loc_index = result.back();
        result.pop_back();

        // Check that we don't get the same item_index back more than once
        assert(completed.find(work_item_index) == completed.end());
        completed.insert(work_item_index);

        // Assert that the supervisor didn't return a work item
        assert(loc_index != 0);

        apply_result(work_item_index,result);
    }

    auto finish = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(finish - start);
    std::cout << "duration: " << (duration.count()*1e-6) << std::endl;

    // Check that all the work was done.
    for(int i=0;i<work_to_be_done;i++) {
        assert(completed.find(i) != completed.end());
    }

    finish_up();
}

In [ ]:
%%writefile easy.mk
image.png : image.pbm
	convert image.pbm image.png

easy.o : easy.cpp
	hpxcxx -c easy.cpp

pbm3.o : pbm3.cpp pbm.hpp
	hpxcxx -c pbm3.cpp

easy : easy.o pbm3.o
	hpxcxx --exe=easy easy.o pbm3.o

image.pbm : ./easy .ALWAYS
	hpxrun.py -t 1 -l 3 ./easy

.ALWAYS:

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/local/lib64
export VAL1=-.4
export VAL2=.6
make -f easy.mk

In [ ]:
%png image.png

In [ ]:
%%writefile sw.cpp
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <iostream>
#include <vector>
#include <math.h>
#include <assert.h>
#include <chrono>
#include <set>
#include <atomic>

using namespace std::chrono;

// Supervisor/Worker parallelism

// The following functions and variable must be defined elsewhere
extern void create_result(int work_item,std::vector<int>& result);
extern void apply_result(int work_item,const std::vector<int>& result);
extern void finish_up();
extern int work_to_be_done;

high_resolution_clock::time_point start, finish;

std::atomic<int> next_work_item(0);
std::atomic<int> completed_work_items(0);

// Called by workers to get an item of work from the supervisor
int get_work() {
    return next_work_item++;
}

HPX_PLAIN_ACTION(get_work, get_work_action);

// Called by workers to send a result to the supervisor 
void send_result(const std::vector<int>& result,int item_index) {
    if(item_index < work_to_be_done) {
        apply_result(item_index, result);
        int n = ++completed_work_items;
        // If I am the one to complete the last work item
        // then I can call finish up.
        if(n == work_to_be_done) {
            auto finish = high_resolution_clock::now();
            auto duration = duration_cast<microseconds>(finish - start);
            std::cout << "duration: " << (duration.count()*1e-6) << std::endl;
            finish_up();
        }
    }
}

HPX_PLAIN_ACTION(send_result, send_result_action);

// The basic unit of work. Does nothing but hand
// args back aa a std::vector<int>.
void do_work(const hpx::naming::id_type& loc, int item_index) {
    while(item_index < work_to_be_done) {
        hpx::future<int> fitem = hpx::async<get_work_action>(loc);
        std::vector<int> result;
        create_result(item_index,result);
        //std::cout << "Work item " << item_index << " is done" << std::endl;
        hpx::async<send_result_action>(loc,result,item_index);
        item_index = fitem.get();
    }
}

HPX_PLAIN_ACTION(do_work, do_work_action);

int main() {
    auto locs = hpx::find_all_localities();

    // The supervisor
    auto here = hpx::find_here();

    // Assert that the supervisor is first in the list
    assert(locs[0] == here);

    // Compute the number of worker localities
    const int num_workers = locs.size() - 1;

    // Make sure we have some workers
    assert(num_workers > 0);

    // The logic below assumes more work than workers
    assert(num_workers < work_to_be_done);

    // Place to hold futures for current worker tasks
    std::vector<hpx::future<void>> data;

    start = high_resolution_clock::now();

    // Load up workers...
    for(int j=0;j<num_workers;j++) {
        int loc_index = 1 + (j % num_workers);
        auto loc = locs.at(loc_index);
        hpx::future<void> f = hpx::async<do_work_action>(loc, here, next_work_item++);
        data.push_back(std::move(f));
    }
    for(int j=0;j<data.size();j++) {
        data.at(j).wait();
    }

}

In [ ]:
%%writefile sw.mk
image.png : image.pbm
	convert image.pbm image.png

sw.o : sw.cpp
	hpxcxx -c sw.cpp

pbm3.o : pbm3.cpp pbm.hpp
	hpxcxx -c pbm3.cpp

sw : sw.o pbm3.o
	hpxcxx --exe=sw sw.o pbm3.o

image.pbm : ./sw .ALWAYS
	hpxrun.py -t 1 -l 3 ./sw

.ALWAYS:

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/local/lib64
export VAL1=-.4
export VAL2=.6
make -f sw.mk

In [ ]:
%png image.png

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/local/lib64
export VAL1=.326
export VAL2=-.53
make -f sw.mk

In [ ]:
%png image.png

In [ ]:
%%writefile fib.cpp
#include <iostream>
#include <vector>

int fib(int n) {
    if(n < 2)
        return n;
    else
        return fib(n-1)+fib(n-2);
}

int work_to_be_done = 14;
extern int work_to_be_done;

void create_result(int n,std::vector<int>& r) {
    r.push_back(fib(n+30));
}
void apply_result(int n,const std::vector<int>& r) {
    ;
}
void finish_up() {
    std::cout << "Fibonacci done" << std::endl;
}

In [ ]:
%%writefile fib.mk
all: fib1 fib2 fib3
	./fib1
	hpxrun.py -t 1 -l 3 ./fib2
	hpxrun.py -t 1 -l 3 ./fib3

fib.o: fib.cpp
	g++ -c fib.cpp

fib1 : ./fib.o ./seq.o
	g++ -o fib1 fib.o seq.o

fib2 : ./fib.o ./easy.o
	hpxcxx --exe=fib2 fib.o easy.o

fib3 : ./fib.o ./superworker.o
	hpxcxx --exe=fib3 fib.o sw.o

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/local/lib64
make -f fib.mk